# 导包

In [16]:
import re
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## 读取

In [17]:
train_df = pd.read_csv(r'./datasets/train.csv')
test_df = pd.read_csv(r'./datasets/test.csv')
gender_df = pd.read_csv(r'./datasets/gender_submission.csv')
print(train_df.shape, test_df.shape, gender_df.shape)

(891, 12) (418, 11) (418, 2)


## 查看

In [18]:
print(train_df.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [19]:
print(test_df.dtypes)

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [20]:
gender_df.dtypes

PassengerId    int64
Survived       int64
dtype: object

In [21]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## 缺失

In [22]:
print(train_df.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [23]:
# 数值类型, 使用均值填充
train_df['Age'].fillna(train_df['Age'].mean(), inplace=True)
# 字符串类型，使用未知填充
train_df['Cabin'].fillna('Unknown', inplace=True)
# 字符串类型，使用未知填充
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
# 查看空值
print(train_df.isnull().sum())

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


## 特征工程

### 特征工程

如果你的特征只有几个离散的类别，而不是一个大的类别集，那么选择哪种方式通常取决于你的数据和机器学习模型的性质。以下是对三种方式的简要比较：

1. **LabelEncoder:**
   - **优点：** 简单，不引入高维度。适用于有序的类别。
   - **注意：** 只适合于有序的类别，因为它会为每个类别分配一个整数标签，模型可能会误以为这些整数之间存在某种有序关系。

   ```python
   from sklearn.preprocessing import LabelEncoder

   label_encoder = LabelEncoder()
   train_df['Category'] = label_encoder.fit_transform(train_df['Category'])
   ```

2. **get_dummies:**
   - **优点：** 适用于无序的、离散的类别。为每个类别创建一个新的二进制列，不引入假设类别之间有序的信息。
   - **缺点：** 可能引入高维度，但对于几个类型的特征通常是可接受的。

   ```python
   category_dummies = pd.get_dummies(train_df['Category'], prefix='Category')
   train_df = pd.concat([train_df, category_dummies], axis=1)
   ```

3. **map:**
   - **优点：** 可以手动指定映射，更加灵活。适用于有序或无序的类别。
   - **缺点：** 需要提前定义映射字典，可能需要对数据进行额外的处理。

   ```python
   category_mapping = {'A': 0, 'B': 1, 'C': 2}
   train_df['Category'] = train_df['Category'].map(category_mapping)
   ```

如果你的特征只有几个类型，而且这些类型之间没有明显的有序关系，通常可以优先考虑使用 `get_dummies` 或 `map`。`get_dummies` 会为每个类别创建一个独热编码列，而 `map` 允许你手动指定映射关系。在这种情况下，选择哪一种通常取决于你更喜欢使用哪种方式，并且哪一种方式更符合你对数据的理解。

In [24]:
train_df['Name'].head()

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
Name: Name, dtype: object

In [25]:
# name特征
train_df['LastName'] = train_df['Name'].apply(lambda x: x.split(',')[0])
train_df['Title'] = train_df['Name'].apply(lambda x: re.search(' ([A-Za-z]+)\.', x).group(1))
train_df['NameLength'] = train_df['Name'].apply(len)
train_df[['LastName', 'Title', 'NameLength']].head(3)

,LastName,Title,NameLength
0,Braund,Mr,23
1,Cumings,Mrs,51
2,Heikkinen,Miss,22


In [26]:
label_encoder = LabelEncoder()
train_df['LastName'] = label_encoder.fit_transform(train_df['LastName'])
train_df['Title'] = label_encoder.fit_transform(train_df['Title'])
train_df['Cabin'] = label_encoder.fit_transform(train_df['Cabin'])

train_df[['LastName', 'Title', 'NameLength']].head(3)

,LastName,Title,NameLength
0,73,12,23
1,136,13,51
2,251,9,22


In [27]:
# sex、Embarked特征
train_df['Sex'] = train_df['Sex'].astype('category')
train_df['Embarked'] = train_df['Embarked'].astype('category')
train_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,LastName,Title,NameLength
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,147,S,73,12,23
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,81,C,136,13,51
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,147,S,251,9,22


## 训练数据

In [28]:
# 将数据分割为特征矩阵和目标变量
X = train_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'LastName', 'Title', 'NameLength']]
y = train_df['Survived']

# 将数据划分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建LightGBM数据集
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=['Sex', 'Embarked'])

In [29]:
## 训练过程

In [30]:
# 定义 LightGBM 模型训练参数

params = {
    "objective": "binary",  # 优化目标：二分类问题
    "metric": "binary_logloss",  # 评估指标：二分类错误率
    "boosting_type": "gbdt",  # 使用梯度提升树算法
    "num_leaves": 15,  # 每棵树的最大叶子节点数，控制树的复杂度
    "learning_rate": 0.1,  # 学习率，控制每一步迭代中模型参数的调整幅度
    "feature_fraction": 0.8,  # 每次迭代中随机选择特征的比例，防止过拟合
    "bagging_fraction": 0.8,  # 每次迭代中随机选择数据的比例，防止过拟合
    "bagging_freq": 5,  # 多少次迭代进行 bagging，增加模型的多样性
    "verbose": 2  # 控制训练过程的输出信息详细程度，设置为 2 可以提供相对详细的训练信息
}

# 应用场景和指导：
# - "objective": 通常根据具体问题选择适当的目标，对于二分类问题，使用 "binary"。
# - "metric": 选择与问题相适应的评估指标，"binary_error" 用于二分类问题的错误率。
# - "boosting_type": 根据数据和问题选择合适的 boosting 类型，"gbdt" 是一种常见的选择。
# - "num_leaves": 调整树的复杂度，较小的值有助于防止过拟合，但可能降低模型拟合能力。
# - "learning_rate": 学习率的选择影响模型收敛速度，过大可能导致震荡，过小可能需要更多迭代。
# - "feature_fraction" 和 "bagging_fraction": 控制每次迭代中随机选择特征和数据的比例，防止过拟合。
# - "bagging_freq": 控制 bagging 的频率，增加模型的多样性，通常选择一个适中的值。
# - "verbose": 控制训练过程的输出信息，调试阶段设置较大的值，生产环境设置较小的值或为 0。


# 将测试集作为验证集
valid_data = lgb.Dataset(X_test, label=y_test)
valid_sets = [valid_data]

model = lgb.train(params, train_data, num_boost_round=100, valid_sets=valid_sets, valid_names=["validation"])

# 在测试集上进行预测
y_pred = model.predict(X_test)
y_pred_binary = [1 if p >= 0.5 else 0 for p in y_pred]

# 计算准确率
accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Accuracy: {accuracy}')

[LightGBM] [Info] Number of positive: 268, number of negative: 444
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.739466
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.301966
[LightGBM] [Debug] init for col-wise cost 0.000104 seconds, init for row-wise cost 0.000231 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000280 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 538
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 11
[LightGBM] [Debug] Use subset for bagging
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.376404 -> initscore=-0.504838
[LightGBM] [Info] Start training from score -0.504838
[LightGBM] [Debug] Re-bagging, using 575 data to train
[LightGBM] [Debug] Trained a tree with leaves = 15 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 15 and depth = 6
[LightGBM] [Debug] Trained a 

### 梯度提升树说明
是不是简单可以理解为嗯决策树增长的过程，比如说当前这一轮决策的过程中，它只生成了当前这一轮的增长，那么这么就这就叫一个弱模型。那所谓集成学习就是将这些每一轮的增长叠加起来，然后构成了一个大的树。这就是基层学习的过程对吗？

ChatGPT
你的理解基本正确，梯度提升的确可以用决策树的增长过程来解释。让我们再强调一下：

弱模型的生成： 在每一轮梯度提升中，弱模型通常是决策树。这棵决策树是为了捕捉当前模型无法很好拟合的模式，它只是对数据进行了一部分的拟合，因此被称为弱学习器。

集成学习的过程： 每一轮的弱模型都在尝试修正上一轮模型的预测误差，而整体模型就是将每一轮的弱模型组合起来的结果。这个组合可以是简单的加法（对回归问题），也可以是一种加权的组合。最终，整体模型的预测是通过将每个弱模型的输出相加而得到的。

基层学习的概念： 在梯度提升中，这种逐轮生成弱模型、每轮弱模型只能拟合一部分模式、然后集成所有弱模型的过程可以被看作是一种基层学习。基层学习强调的是通过多次迭代，每次迭代生成一个弱模型，并通过集成来提升整体模型性能的过程。

所以，你可以将梯度提升看作是基层学习的一种实现方式，通过逐步修正预测误差，每一轮生成一个弱模型，最终构成一个强大的集成模型。这种集成学习的思想也是梯度提升的核心。